In [3]:
import pandas as pd
PATH ="data/"
FILE_NAME = "haberman.csv"

columns = ["age","year", "nodes", "class"]

df = pd.read_csv(PATH + FILE_NAME,header=None, names=columns)

In [4]:
df

,age,year,nodes,class
0,30,64,1,1
1,30,62,3,1
2,30,65,0,1
3,31,59,2,1
4,31,65,4,1
...,...,...,...,...
301,75,62,1,1
302,76,67,0,1
303,77,65,3,1
304,78,65,1,2
